In [2]:
import utils

We load our original JSON dataset and transform it the same way we did for the Cassandra database.

In [3]:
merged_df = utils.prepare_dataset(utils.DATASET_PATH)

Let’s now count the final number of records. 😊

In [ ]:
merged_df.to_json('merged_df.jsonl', orient='records', lines=True)